## 11. 통계적 가설검정
### - 통계적 가설검정
### - 기본적인 가설검정
### - 2표본 문제에 관한 가설검정

* 통계적 가설검정 -> 모집단의 모수에 관한 가설을 세우고 표본의 정보를 이용해 그 가설을 검증하는 기법

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

%precision 3
%matplotlib inline
np.random.seed(1111)

In [2]:
# 테스트 데이터 준비
df = pd.read_csv("ch11_potato.csv")
sample = np.array(df['무게'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [3]:
# 표본평균 구하기
s_mean = np.mean(sample)
s_mean

128.4507142857143

#### 1. 통계적 가설검정
* 모집단의 모수에 관하여 두 가지 가설을 세우고, 표본으로부터 계산되는 통계량을 이용하여 어느 가설이 옳은지 판단하는 통계적인 방법
* 간단하게 가설검정 or 검정이라고도 부름
* 통계적 가설검정의 기본
  - 감자튀김 예시를 통해 확인
    + 확인하고 싶은 것은 모평균이 130g보다 작은지 여부, 감자튀김의 모집단이 정규분포를 따르고 모분산=9임을 알고 있다고 전제
    + 표본평균이 128.451g인 경우 5% 확률로 일어나는 운나쁜 케이스라고 생각할 수도
    + 하지만 가설검정을 통해 가정이 이상하다고 판단하여 '원래 모평균이 130g보다 작은 게 아닐까?'라는 의문이 발생하고, 이 의문에서 '모평균이 130g보다 작다'라는 결론을 내리는 것이 가설검정의 큰 흐름
    + 가설검정에서는 모수에 관한 두 가설, 귀무가설(null hypothesis)과 대립가설(alternative hypothesis)를 사용
    + 대립가설 -> 주장하고 싶은 가설 ('차이가 있다','효과가 있다' 등의 내용) ($H_1$)
    + 귀무가설 -> '차이가 없다', '효과가 없다'는 내용 ($H_0$)
    + 이 두 가설을 검정하기 위해 표본으로부터 통계량을 계산하여 가설검정을 수행
    + 결론 -> '귀무가설을 기각한다(reject the null hypothesis)' or '귀무가설을 채택한다(accept the null hypothesis)' 중 하나
    + 귀무가설을 기각 -> 귀무가설은 옳지 않다
    + 귀무가설을 채택 -> 귀무가설이 옳지 않다고 말할 수 없다 (귀무가설이 옳은지 여부를 알 수 없다고 보류하는 결론)
    + 귀무가설의 기각과 채택은 귀무가설의 가정을 바탕으로 했을 때 표본으로부터 계산되는 통계량이 드문 값인지 여부로 결정
    + 드문 값을 얻으면 그것은 우연이 아니라 어떤 의미 있는 값이라고 생각하여 귀무가설 기각
    + 그렇지 않다면 귀무가설 채택
    + 여기서 우연이 아니라 어떤 의미가 있는 것을 '유의하다(significant)'라고 함
    + 감자 예시로 돌아와서...
      * 여기서 대립가설은 '모평균은 130g보다 작다'가 됨
      * 귀무가설은 '모평균은 130g이다'가 됨
      * 얻을 수 있는 결론 -> '모평균은 130g보다 작다'(귀무가설 기각) or '모평균은 130g보다 작다고 말할 수 없다'(귀무가설 채택)
      * 귀무가설 '모평균은 130g이다'라는 가정을 바탕으로 했을 때 표본평균이 128.451g이 되는 것은 유의하므로 귀무가설은 기각됨
      * 정리하자면... 표본평균 128.681g보다 작다면 귀무가설 기각, 크다면 귀무가설 채택
      * 귀무가설이 기각되는 구간 = 기각역(rejection region), 귀무가설이 채택되는 구간 = 채택역(acceptance region)
      * 기각역에 들어갈 확률 = 유의수준(level of significance), 경게선상의 값 = 임곗값(criticial value)
      * 유의수준은 어느 정도의 확률로 발생하는 사건을 드물다고 인식할 것인지를 설정 (관습적으로 1% or 5% 설정)
      * 검정에 사용되는 통계량 = 검정통계량(test statistic)
      * 감자튀김 예시에선 유의수준 5% 설정, 검정통계량으로 표본평균 사용, 임곗값은 128.681이 됨
      * 검정통계량이 임곗값보다 작을 때, 임곗값보다 왼쪽에 있는 영역의 면적은 유의수준, 검정통계량보다 왼쪽에 있는 영역의 면적은 p값(p-value)가 됨
      * 가설검정은 검정통계량과 임곗값의 비교가 아니라 p값과 유의수준의 비교로 수행할 수도 있음
      * 이 경우 p값이 유의수준보다 작을 때 -> 귀무가설 기각, 그렇지 않을 때 -> 귀무가설 채택
      * p값을 기준으로 수행하는 가설검정 흐름 정리: 가설을 세운다 --> 유의수준을 결정한다 --> 검정통계량을 계산한다 --> p값을 계산한다 --> p값이 유의수준보다 큰가? --> (yes) 귀무가설을 채택, (no) 귀무가설을 기각
* 단측검정과 양측검정
  - '모평균은 130g이 아니다.'라는 대립가설로 가설검정을 수행할 수도 있음 --> 모평균이 130g보다 작은 경우 + 130g보다 큰 경우 => 양측검정
  - '모평균은 130g보다 작다.'라는 대립가설로 가설검정 진행 => 단측검정
  - 단측검정과 양측검정의 기각역이 다름 --> 동일한 유의수준 $\alpha$의 검정이라도 단측검정 쪽이 더 넓음 => 단측검정이 양측검정보다 귀무가설을 기각하기 쉬움
* 가설검정의 두 가지 오류
  - 표본을 이용하여 확률적으로 결론을 유도하기 때문에 발생
  - 제1종 오류: 귀무가설이 옳을 때, 귀무가설을 기각하는 오류
    + 실제로 '평균이 130g'인데도 '평균은 130g보다 작다'라는 결론을 내리는 상황
    + 본래 검출하지 말아야 할 것을 검출한 것 => 오탐(false positive)
  - 제2종 오류: 대립가설이 옳을 때, 귀무가설을 채택하는 오류
    + 실제로 '모평균이 130g보다 작다'인데도 '모평균은 130g보다 작다'라는 결론을 얻을 수 없는 상황
    + 검출해야 하는 것을 검출하지 못한 것 => 미탐(false negative)

In [4]:
# P(표본평균 <= x) = 0.05를 만족하는 x를 구하기
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95) # 표본평균이 128.681g 이하의 무게가 되는 것은 5% 확률로 발생한다는 것

128.68118313069039

* 감자튀김 예시로 가설검정 흐름 확인
  - 귀무가설 '모평균은 130g'이라고 가정한 것을 바탕으로 14개의 감자튀김은 서로 독립이고 $N(130,9)$를 따르며 표본평균은 $N(130,{9 \over 14})$를 따른다.
  - 검정통계량은 (일반화하여 설명할 수 있도록) 표본평균 $\bar{X}$를 표준화한 $Z = (\bar{X} -130)/\sqrt{{9 \over 14}}$을 활용
  - 표준화를 통해 상위 $100\alpha$%점을 $z_{\alpha}$로 나타낼 수 있고 임곗값은 $P((\bar{X} -130)/\sqrt{{9 \over 14}} \leq x) = 0.05$를\
  만족하는 $x$를 구할 수 있음 (즉 $x=z_{0.95}$)
  - 그러므로\
  $(\bar{X} -130)/\sqrt{{9 \over 14}} < z_{0.95}$이면 귀무가설 기각\
  $(\bar{X} -130)/\sqrt{{9 \over 14}} \geq z_{0.95}$이면 귀무가설 채택

In [5]:
# 검정통계량 구하기
z = (s_mean -130) / np.sqrt(9/14)
z

-1.932298779026813

In [6]:
# 임곗값 구하기
rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

* 검정통계량 < 임곗값 ==> 귀무가설은 기각되고 평균은 130g보다 작다는 결론 (검정통계량으로 표본평균을 사용해도 동일한 결론)

In [7]:
# p값을 활용한 가설검정
rv.cdf(z)

0.026661319523126635

* p값이 0.027로 유의수준 0.05보다 작은 값 ==> 귀무가설 기각

In [8]:
# 양측검정 수행
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [9]:
# 양측검정이므로 임곗값은 표준정규분포의 95% 구간에 따라 구할 수 있음
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

* 검정통계량이 채택역에 들어 있음 => 양측검정에서는 귀무가설이 기각되지 않음

In [10]:
# 양측검정의 p값 구하기
# 상단,하단 양쪽 면적을 고려할 필요가 있으므로 누적밀도함수의 값을 2배로 함
rv.cdf(z)*2

0.05332263904625327

* p값 > 0.05(유의수준) => 귀무가설은 기각되지 않음

In [11]:
# 제1종 오류가 어느 정도의 비율로 발생하는지 시뮬
# 실제로 '평균은 130g'인 상황을 고려하고 있으므로 모집단의 확률분포는 N(130,3^2)
rv = stats.norm(130,3)

In [13]:
# 이 모집단에서 14개의 표본을 추출하여 가설검정을 수행하는 작업을 1만번 시행
# 그리고 제1종 오류를 범하는 비율 계산
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14),2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
cnt / n_samples

0.053

* 약 5%의 비율로 '130g보다 작다'라고 잘못 탐지하는 것을 알 수 있음
* 제1종 오류를 범하는 확률 => 위험률($\alpha$)
* 위험률은 유의수준과 일치하므로 분석가가 제어할 수 있는 확률
* 제1종 오류가 발생하는 확률을 1%로 하고 싶다면 분석가는 유의수준 1%에서 가설검정 수행하면 됨

In [14]:
# 제2종 오류가 어느 정도의 비율로 발생하는지 시뮬
# 평균이 128g으로 설정되어 있다고 가정 -> 모집단의 확률분포는 N(128,3^2)
rv = stats.norm(128,3)

In [15]:
# 제2종 오류를 범하는 비율 계산
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14),2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
cnt / n_samples

0.197

* 약 20%의 비율로 미탐 발생
* 제2종 오류를 범하는 확률 => $\beta$, 1-$\beta$ => 검정력(power)
* 동일하게 '모평균이 130g보다 작다'라는 상황이라도 평균이 120g으로 설정되면 미탐이 발생할 확률은 낮아짐 -> $\beta$는 모집단의 정보에 의존
* 본래 모집단의 정보는 알 수 없으므로 $\beta$는 분석가가 제어할 수 없는 확률임

#### 2. 기본적인 가설검정 (양측검정으로 진행)
* 정규분포의 모평균에 대한 검정: 모분산을 알고 있는 경우
  - $X_1,X_2,\dots,X_n {\sim^{iid}} N(\mu,\sigma^2)$일 떄, 모평균 $\mu$에 관한 유의수준 $\alpha$의 양측검정
    + 귀무가설: $\mu$ = $\mu_0$
    + 대립가설: $\mu$ $\neq$ $\mu_0$
  
  은 검정통계량으로 $Z = ({\bar{X}} - {\mu_0})/{\sqrt{\sigma^2} \over {n}}$을 사용하여
    + $Z < z_{1-{\alpha}/2}$ 또는 $z_{\alpha/2} < Z$라면, 귀무가설을 기각
    + $z_{1-{\alpha}/2} \leq {Z} \leq z_{\alpha/2}$라면, 귀무가설을 채택
   
  으로 수행
  
* 정규분포의 모분산에 대한 검정 (모분산이 어떤 값이 아닌 것을 주장하기 위한 검정)
  - $X_1,X_2,\dots,X_n {\sim^{iid}} N(\mu,\sigma^2)$일 떄, 모분산 $\sigma^2$에 관한 유의수준 $\alpha$의 양측검정\
    + 귀무가설: $\sigma^2$ = ${\sigma_0}^2$
    + 대립가설: $\sigma^2$ $\neq$ ${\sigma_0}^2$
  
  은 검정통계량으로 ${Y} = {{(n-1){s^2}} \over {\sigma^2}} $을 사용하여
    + $Y < \chi_{{1-\alpha}/2}^2(n-1)$ 또는 $\chi_{{\alpha}/2}^2(n-1) < Y$라면, 귀무가설을 기각
    + $\chi_{{1-\alpha}/2}^2(n-1) \leq {Y} \leq \chi_{{\alpha}/2}^2(n-1)$라면, 귀무가설을 채택
   
  으로 수행
  
* 정규분포의 모평균에 대한 검정: 모분산을 모르는 경우
  - 모분산을 알지 못하는 상황에서 정규분포의 모평균에 대한 검정 => 1표본 t검정(1-sample t-test)
  - t검정통계량($t = (\bar{X}-{\mu_0})/{\sqrt{s^2} \over {n}}$)을 검정통계량으로 사용
  
  - $X_1,X_2,\dots,X_n {\sim^{iid}} N(\mu,\sigma^2)$일 떄, 모평균 $\mu$에 관한 유의수준 $\alpha$의 양측검정\
    + 귀무가설: $\mu$ = $\mu_0$
    + 대립가설: $\mu$ $\neq$ $\mu_0$
    
  은 검정통계량으로 $t = (\bar{X}-{\mu_0})/{\sqrt{s^2} \over {n}}$을 사용하여
    + $t<t_{1-{\alpha/2}}(n-1)$ 또는 $t_{\alpha/2}<t$라면, 귀무가설을 기각
    + $t_{1-{\alpha/2}}(n-1) \leq {t} \leq t_{\alpha/2}$라면, 귀무가설을 채택
    
  으로 수행

In [16]:
# 정규분포의 모평균에 대한 검정: 모분산을 알고 있는 경우
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
        
    if z<0:
        p = rv.cdf(z)*2
    else:
        p = (1-rv.cdf(z))*2
    print(f'p값은 {p:.3f}')

In [17]:
pmean_test(sample, 130, 9)

귀무가설을 채택
p값은 0.053


In [18]:
# 정규분포의 모분산에 대한 검정
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample,ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
        
    if y < rv.isf(0.5):
        p = rv.cdf(y)*2
    else:
        p = (1 - rv.cdf(y))*2
    print(f'p값은 {p:.3f}')

In [19]:
pvar_test(sample,9)

귀무가설을 채택
p값은 0.085


In [21]:
# 정규분포의 모평균에 대한 검정: 모분산을 모르는 경우
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)
    
    t = (s_mean - mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
        
    if t<0:
        p = rv.cdf(t)*2
    else:
        p = (1-rv.cdf(t))*2
    print(f'p값은 {p:.3f}')

In [22]:
pmean_test(sample,130)

귀무가설을 채택
p값은 0.169


In [23]:
# 1표본 t검정 -> scipy.stats의 ttest_1samp 함수로 구현할 수 있음
t,p = stats.ttest_1samp(sample,130)
t,p

(-1.4551960206404198, 0.16933464230414275)

#### 3. 2표본 문제에 관한 가설검정
* 두 모집단에 대한 검정 => 2표본 문제(two-sample problem)
* 대응표본: 두 데이터에서 서로 대응하는 동일한 개체에 대해 각각 다른 조건으로 측정한 것 (ex. 피검자에게 약을 투여하기 전후에 측정한 혈압)
* 독립표본: 데이터에 대응이 없다는 것 (ex. A조 학생의 시험점수 & B조 학생의 시험점수)
* (정규분포 가정 가능, 대응표본) = 대응비교 t검정(paired t-test)
  - 대응하는 데이터가 있고, 데이터 차이에 정규분포를 가정할 수 있는 경우 평균값 차이에 대한 검정
* (정규분포 가정 가능, 독립표본) = 독립비교 t검정(independent t-test)
  - 대응하는 데이터가 없고 독립된 2표본 모집단에 정규분포를 가정할 수 있는 경우 평균값 차이에 대한 검정
* (정규분포 가정 불가능, 대응표본) = 윌콕슨의 부호순위검정
* (정규분포 가정 불가능, 독립표본) = 만$\cdot$위트니의 U검정

In [24]:
# 대응비교 t검정
# 근력운동과 집중력
training_rel = pd.read_csv('E:/jupyter/누구나 파이썬 통계분석/ch11_training_rel.csv')
print(training_rel.shape)
training_rel.head()

(20, 2)


,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


* 근력운동이 집중력을 향상시키는 효과가 있는지 여부 -> 근력운동 전과 근력운동 후의 집중력 테스트의 평균 점수를 비교
* $\mu_before$ = 근력운동 전의 집중력 테스트 평균 점수, $\mu_after$ = 근력운동 후의 집중력 테스트 평균 점수
  - 귀무가설: $\mu_{after}$ - $\mu_{before}$ = 0
  - 대립가설: $\mu_{after}$ - $\mu_{before}$ $\neq$ 0

In [25]:
training_rel['차'] = training_rel['후'] - training_rel['전']
training_rel.head()

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


* 근력운동이 집중력 테스트에 끼치는 영향이 없다면, 그 차이는 임의로 분산되어 평균이 0인 분포가 될 것
* 차이의 평균 = $\mu_{diff}$
  - 귀무가설: $\mu_{diff}$ = 0
  - 대립가설: $\mu_{diff}$ $\neq$ 0
* 더 나아가 그 차이가 각각 독립이고 동일한 정규분포를 따르고 있다고 가정할 수 있으면,
* 이 검정은 모분산을 모르는 경우의 정규분포의 모분산에 대한 검정(1표본 t검정)으로 귀착될 수 있음

In [26]:
t,p = stats.ttest_1samp(training_rel['차'],0)
p

0.04004419061842953

* p값이 유의수준보다 작음 => 귀무가설 기각

In [27]:
# before, after의 데이터로 동일한 검정 수행 가능
t,p = stats.ttest_rel(training_rel['후'],training_rel['전'])
p

0.04004419061842953

In [28]:
# 독립비교 t검정
training_ind = pd.read_csv('E:/jupyter/누구나 파이썬 통계분석/ch11_training_ind.csv')
print(training_ind.shape)
training_ind.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


* 근력운동이 집중력을 향상하는 효과가 있는지 -> A 학생의 학급과 B 학생의 학급에서 진행한 집중력 테스트에서 평균 점수를 비교
* $\mu_1$ = A 학생의 학급 평균점수, $\mu_2$ = B 학생의 학급 평균점수일 때
  - 귀무가설: $\mu_1 - \mu_2$ = 0
  - 대립가설: $\mu_1 - \mu_2$ $\neq$ 0
* 독립표본이므로 이번에는 차이를 구해도 아무 의미가 없음
* A 학생의 학급 표본과 B 학생의 학급 표본은 별개의 모집단에서 추출된 것이고 이러한 모집단에 정규분포를 가정하면
  - A 학생의 학급 점수 $X_1,X_2,\dots,X_{n_1} \sim N(\mu_1,{\sigma_1}^2)$
  - B 학생의 학급 점수 $Y_1,Y_2,\dots,Y_{n_1} \sim N(\mu_2,{\sigma_2}^2)$ 이라 할 수 있음
* 이러한 가정을 바탕으로 검정통계량은
  - t = ${(\bar{X} - \bar{Y}) - (\mu_1 - \mu_2)} \over \sqrt{{{s_1}^2 \over {n_1}} + {{s_2}^2 \over {n_2}}}$를 사용
*